# Word2Vec Skip-gram
Neste notebook, estaremos treinando uma das variantes do Word2Vec, o Skip-Gram. Para isso, vamos construir uma rede neural com apenas uma hidden layer. Sendo assim, a nossa arquitetura vai operar como um modelo de linguagem, ou seja, a tarefa que a rede irá realizar é "dada uma palavra *X*, tente prever a próxima palavra *Y*". 
Porém, para extrair os nossos word vectors, usaremos apenas os pesos da nossa camada escondida. Assim, o nosso objetivo final será aprender os pesos da nossa hidden layer. 

Nesse cenário, cada um dos elementos do nosso word vector funcionará como uma "feature" da nossa palavra representada, ou seja, cada uma dessas dimensões é responsável por uma característica da composição semântica dessa palavra. Como estamos mapeando os seus significados a partir do contexto em que a palavra está inserida, a ideia principal que o word2vec utiliza, é a de que palavras que acontecem em *contextos* semelhantes também possuem *significados* semelhantes. 

#### Importando bibliotecas 

In [1]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import re

#### Lendo o nosso arquivo 

In [2]:
f = open("biblia-em-txt.txt", "r")
text = f.read()

In [3]:
text[:500]

'BÍBLIA SAGRADA\nTradução: João Ferreira de Almeida\nEdição Revista e Corrigida\n \t\nANTIGO TESTAMENTO\n\nGÊNESIS\n GÊNESIS 1\n1 No princípio criou Deus os céus e a terra.\n2 A terra era sem forma e vazia; e havia trevas sobre a face do abismo, mas o Espírito de Deus pairava sobre a face das águas.\n3 Disse Deus: haja luz. E houve luz.\n4 Viu Deus que a luz era boa; e fez separação entre a luz e as trevas.\n5 E Deus chamou à luz dia, e às trevas noite. E foi a tarde e a manhã, o dia primeiro.\n6 E disse Deus:'

#### Tokenizando o texto 

In [4]:
def tokenize_sentece(sent):
    tokenized = word_tokenize(sent)
    tokenized = [token.lower() for token in tokenized if token.isalpha()]
    return tokenized

def tokenize_text(text):
    sentences = sent_tokenize(text)
    tokenized_senteces = [tokenize_sentece(sent) for sent in sentences]
    return tokenized_senteces

In [5]:
sent_tokens = tokenize_text(text)

In [6]:
sent_tokens[0:2]

[['bíblia',
  'sagrada',
  'tradução',
  'joão',
  'ferreira',
  'de',
  'almeida',
  'edição',
  'revista',
  'e',
  'corrigida',
  'antigo',
  'testamento',
  'gênesis',
  'gênesis',
  'no',
  'princípio',
  'criou',
  'deus',
  'os',
  'céus',
  'e',
  'a',
  'terra'],
 ['a',
  'terra',
  'era',
  'sem',
  'forma',
  'e',
  'vazia',
  'e',
  'havia',
  'trevas',
  'sobre',
  'a',
  'face',
  'do',
  'abismo',
  'mas',
  'o',
  'espírito',
  'de',
  'deus',
  'pairava',
  'sobre',
  'a',
  'face',
  'das',
  'águas']]

### Mapeando as palavras do nosso vocabulário
Antes de partirmos para a arquitetura da rede em si, precisamos preparar o nosso corpus. 

A primeira parte é construir um dicionário com todas as palavras do nosso texto - `vocab` - e os seus índices:

In [7]:
def make_vocab(sent_tokenized):
    i = 0
    vocab = {}
    for sent in sent_tokenized:
        for token in sent:
            if token in vocab:
                continue
            else:
                vocab[token] = i
                i+=1
    return vocab

In [8]:
vocab = make_vocab(sent_tokens)

In [9]:
V = len(vocab)
print(V)

24714


Aqui, podemos observar que o nosso vocabulário possui 24714 palavras únicas. 

Agora, precisamos gerar as palavras centrais com as suas respectivas palavras de contexto. 

Para o modelo Skip-Gram, devemos montar uma tupla para a palavra central com cada uma das suas palavras de contexto. 

Observe o exemplo: 
![](http://mccormickml.com/assets/word2vec/training_data.png)

In [10]:
def get_windows(words, C):
    i = 0
    while i < len(words): 
        center_word = words[i] 
        if i < C:
            context_words = words[0:i] + words[(i+1):(i+C+1)]
        else:
            context_words = words[(i - C):i] + words[(i+1):(i+C+1)]
        
        
        for w in range(len(context_words)):
            context_word = context_words[w]
            yield center_word, context_word
        
        i += 1

In [11]:
def make_dataset(sent_tokens, vocab, V, C):
    '''
    inputs:
        sent_tokens: lista de listas tokenizadas 
        vocab -- dicionário com {word:index}
        V -- tamanho do vocabulário
        C -- tamanho da janela de contexto
    
    outputs: tupla (x,y), em que x=palavra central e y=palavra de contexto
    '''
    
    windows_and_target = []
    for sentence in sent_tokens:
        for x, y in get_windows(sentence,C):
            windows_and_target.append((x,y))
        
    return windows_and_target

In [12]:
exemplo = text[120:162]
exemplo

'No princípio criou Deus os céus e a terra.'

In [13]:
sent = tokenize_text(exemplo)
data_exemplo = make_dataset(sent, vocab, V, C=2)
data_exemplo

[('no', 'princípio'),
 ('no', 'criou'),
 ('princípio', 'no'),
 ('princípio', 'criou'),
 ('princípio', 'deus'),
 ('criou', 'no'),
 ('criou', 'princípio'),
 ('criou', 'deus'),
 ('criou', 'os'),
 ('deus', 'princípio'),
 ('deus', 'criou'),
 ('deus', 'os'),
 ('deus', 'céus'),
 ('os', 'criou'),
 ('os', 'deus'),
 ('os', 'céus'),
 ('os', 'e'),
 ('céus', 'deus'),
 ('céus', 'os'),
 ('céus', 'e'),
 ('céus', 'a'),
 ('e', 'os'),
 ('e', 'céus'),
 ('e', 'a'),
 ('e', 'terra'),
 ('a', 'céus'),
 ('a', 'e'),
 ('a', 'terra'),
 ('terra', 'e'),
 ('terra', 'a')]

In [14]:
dataset = make_dataset(sent_tokens, vocab, V, C=2)

## Implementação com Numpy
**Rede Neural para o modelo Skip-Gram:**

![](http://mccormickml.com/assets/word2vec/skip_gram_net_arch.png)



A rede pode ser descrita pelas seguintes equações: 
<br>
$$Z_1 = W_1X + b_1$$
$$Z_2 = W_1h + b_2$$
$$ŷ = softmax(Z_2)$$
<br>

Em que a função softmax pode ser definida:
$$softmax(x_i) = \frac{e^{x_i}}{\sum_{k=1}^N e^{x_k}}$$

Nossa função de custo é a entropia cruzada, que é definida da seguinte forma:
$$J = - \sum_{k=1}^V y_k \ln\hat{y}_k$$
Em que V denota o número de palavras no vocabulário. <br>


In [15]:
def word_to_one_hot_vector(word, vocab, V):
    '''
    inputs:
    word -- string
    vocab -- dicionário com {word:index}
    V -- tamanho do vocabulário
    
    output: vetor one-hot da palavra 
    '''
    one_hot_vector = np.zeros(V)
    one_hot_vector[vocab[word]] = 1
    
    return one_hot_vector

In [16]:
def get_batch(raw_batch, vocab, V, batch_size):
    '''
    inputs:
        raw_batch --  lista de tuplas (x,y)
        vocab -- dicionário com {word:index}
        V -- tamanho do vocabulário
        batch_size -- tamanho da batch   
    '''
    
    batch_x = []
    batch_y = []
    for x, y in raw_batch:
        while len(batch_x) < batch_size:
            batch_x.append(word_to_one_hot_vector(x, vocab, V))
            batch_y.append(word_to_one_hot_vector(y, vocab, V))
        else:
            return np.array(batch_x).T, np.array(batch_y).T

In [17]:
def initialize_params(V,emb_size):
    '''
    inputs:
        V -- tamanho do vocabulário
        emb_size -- número de dimensões do embedding 
    outputs: W1, W2, b1, b2
    '''
    W1 = np.random.rand(emb_size,V)
    W2 = np.random.rand(V,emb_size)
    b1 = np.random.rand(emb_size,1)
    b2 = np.random.rand(V,1)
    
    return W1, W2, b1, b2

In [18]:
def softmax(z):
    e_z = np.exp(z)
    sum_e_z = np.sum(e_z, axis=0) 
    soft = e_z / sum_e_z
    return soft

In [25]:
def forward(x, W1, W2):
    z1 = W1@x 
    h = z1
    z2 = W2@z1 
    yhat = softmax(z2)
    return yhat, h

In [20]:
def compute_cost(y, yhat, batch_size):
    logprobs = y * np.log(yhat)   
    cost = - (1/batch_size) * np.sum(logprobs)
    cost = np.squeeze(cost)
    return cost

In [23]:
def back_prop(x, yhat, y, h, W1, W2, batch_size):
    grad_W1 = (1/batch_size) * np.dot(np.dot(W2.T,(yhat - y)), x.T)
    grad_W2 = (1/batch_size) * np.dot((yhat - y), h.T)
    return grad_W1, grad_W2

In [41]:
BATCH_SIZE = 256
EPOCHS = 1
ALPHA = 0.001
emb_size = 300


W1, W2, b1, b2 = initialize_params(V,emb_size)
for i in range(EPOCHS):
    for j in range(int(len(dataset)/BATCH_SIZE)):
        x,y = get_batch(dataset, vocab, V, BATCH_SIZE)
        yhat, h = forward(x, W1, W2)
        cost = compute_cost(y, yhat, BATCH_SIZE)
        if j%200 == 0:
            print(f"iteration: {j}| epoch {i+1}| cost: {cost}")
        grad_W1, grad_W2 = back_prop(x, yhat, y, h, W1, W2, BATCH_SIZE)
        W1 -= ALPHA*grad_W1
        W2-= ALPHA*grad_W2

iteration: 0| epoch 1| cost: 15.63539376762904
iteration: 200| epoch 1| cost: 0.11261365110622698
iteration: 400| epoch 1| cost: 0.03005449474070234
iteration: 600| epoch 1| cost: 0.017110867542549264
iteration: 800| epoch 1| cost: 0.011929401257448558
iteration: 1000| epoch 1| cost: 0.00914774719287372
iteration: 1200| epoch 1| cost: 0.007414469894138136
iteration: 1400| epoch 1| cost: 0.006231658699166231
iteration: 1600| epoch 1| cost: 0.005373353877290556
iteration: 1800| epoch 1| cost: 0.004722285082881849
iteration: 2000| epoch 1| cost: 0.004211570077451198
iteration: 2200| epoch 1| cost: 0.0038002906637602182
iteration: 2400| epoch 1| cost: 0.0034620143313829287
iteration: 2600| epoch 1| cost: 0.003178908330699638
iteration: 2800| epoch 1| cost: 0.0029385069688249075
iteration: 3000| epoch 1| cost: 0.0027318346734249792
iteration: 3200| epoch 1| cost: 0.002552264975060238
iteration: 3400| epoch 1| cost: 0.002394799509667702
iteration: 3600| epoch 1| cost: 0.0022555971308617505
i

In [42]:
embeddings = (W1.T + W2)

In [43]:
encoded_embeddings = {}
for word, i in vocab.items():
    encoded_embeddings[word] = list(embeddings[i])

In [44]:
import json
with open(f'bible_embeddings_skipgram_300.json', 'w') as file:
    file.write(json.dumps(encoded_embeddings)) 

## Testando os embeddings

In [51]:
import json
with open('bible_embeddings_skipgram_300.json') as json_file:
    emb = json.load(json_file)

In [52]:
def cosine_similarity(x,y):
    return np.dot(x.T,y)/(np.linalg.norm(x)*np.linalg.norm(y))

def find_most_similar(x, word_emb):
    similar = {}
    for word, vec in word_emb.items():
        sim = cosine_similarity(np.array(word_emb[x]),np.array(vec))
        if word != x:
            similar[word] = sim
    similar =  {k: v for k, v in sorted(similar.items(), key=lambda item: item[1], reverse=True)}
    most_10_similar = list(similar.keys())[:10]
    most_similar = [(x, similar[x]) for x in most_10_similar]
    return most_similar